In [1]:
from utils import *
from dataset import *
from train import * 

In [2]:
torch_fix_seed(19981303)

In [3]:
def get_self_pretrain_model(index=1000):
    base_encoder = torchvision.models.resnet50(weights=None)
    base_encoder.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    origin_dim_mlp = base_encoder.fc.in_features
    base_encoder.fc = None
    temp = torch.load(f'./checkpoints/CEM_ALL_CHECK_{index}_Epoch.pth')['state_dict']
    state_dict = {}
    for k, v in temp.items():
        if 'encoder_q' in k:
            if 'fc' not in k:
                state_dict['.'.join(k.split('.')[1:])] = v
    base_encoder.load_state_dict(state_dict)
    base_encoder.fc = torch.nn.Linear(origin_dim_mlp, 512)
    return base_encoder

In [5]:
top_k_candidates = (20, 30, 40)
k = 7
temps = 0.7
momentums = 0.99
k_value = 64
parameters = {'epochs_pretrain_model': (600, 800)}
# parameters = {'pretrain_model': ['CEM']}
train_metrics = HistoryRecorder(['Train Loss', 'Train Acc', 'Val Loss', 'Val Acc'], list(parameters.keys()))
parameters = list(itertools.product(*parameters.values()))

for i, parameter in enumerate(parameters):

    ### custom part to get parameters
    pretrain_model = parameter[0]
    ### END
    
    for j, images in enumerate(k_fold_train_validation_split(ORIGINAL_IMAGE, TARGET_IMAGE, k)):
        train_dataset = SingleChannelNDIDatasetContrastiveLearningWithAug(images, False)
        val_dataset = SingleChannelNDIDatasetContrastiveLearningWithAug(images, True)
        train_iter = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
        val_iter = DataLoader(val_dataset, batch_size=len(val_dataset))
        model = get_self_pretrain_model(index=pretrain_model)
        model = MoCo(model, dim=512, K=k_value, T=temps, m=momentums, mlp=True, customized_model=True)
        device = torch.device('cuda:0')
        criterion = nn.CrossEntropyLoss().cuda(device)
        optimizer = torch.optim.SGD(model.parameters(), lr=0.015, momentum=0.9, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=100)
        start_time = time.time()
        print(f'Parameter Index: {i} / {len(parameters)}, Fold Index: {j} / {k}')
        metrics = train_moco_return_metrics_top_k(model, train_iter, val_iter, criterion, optimizer, 100, device,
                                                    tested_parameter=parameter, k_candidates=top_k_candidates, scheduler=scheduler)
        end_time = time.time()
        train_metrics.cal_add(metrics)
train_metrics.cal_divide(k)

Parameter Index: 0 / 2, Fold Index: 0 / 7
Epoch 1, Train_Loss 3.9181329250335692, Val_loss 4.2900390625
Train_acc_top_20 0.1313	Train_acc_top_30 0.1812	Train_acc_top_40 0.2562	
Val_acc_top_20 0.0	Val_acc_top_30 0.04	Val_acc_top_40 0.08	
Epoch 2, Train_Loss 4.118141841888428, Val_loss 4.2642669677734375
Train_acc_top_20 0.1375	Train_acc_top_30 0.1812	Train_acc_top_40 0.25	
Val_acc_top_20 0.0	Val_acc_top_30 0.12	Val_acc_top_40 0.12	
Epoch 3, Train_Loss 4.056149673461914, Val_loss 4.285374164581299
Train_acc_top_20 0.125	Train_acc_top_30 0.175	Train_acc_top_40 0.225	
Val_acc_top_20 0.0	Val_acc_top_30 0.12	Val_acc_top_40 0.17	
Epoch 4, Train_Loss 3.9555782079696655, Val_loss 4.179591655731201
Train_acc_top_20 0.1187	Train_acc_top_30 0.1938	Train_acc_top_40 0.2625	
Val_acc_top_20 0.04	Val_acc_top_30 0.08	Val_acc_top_40 0.12	
Epoch 5, Train_Loss 3.851576828956604, Val_loss 3.97583270072937
Train_acc_top_20 0.1313	Train_acc_top_30 0.1875	Train_acc_top_40 0.2125	
Val_acc_top_20 0.17	Val_acc_to

KeyboardInterrupt: 